In [1]:
from symred.chem import xyz_from_pubchem
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator
from symred.chem import Draw_molecule

In [2]:
def build_stretched_H2O(bond_length):
    
    angle_degrees=104
    
    angle_rad= np.deg2rad(angle_degrees)
    
    xyz_str = '3\n \nO\t0\t0\t0\n'
    
    # first H
    xyz_str+=f'H\t{bond_length}\t0\t0\n'
    
    # second H
    x_position = bond_length*np.cos(angle_rad)
    y_position = bond_length*np.sin(angle_rad)
    
    if not np.isclose(np.linalg.norm([x_position, y_position]), bond_length):
        raise ValueError('bond length not correct')
    
    xyz_str+=f'H\t{x_position}\t{y_position}\t0'
    
    return xyz_str
    
    

In [3]:
def build_N2(bond_length):    
    xyz_str = f'2\n \nN\t0\t0\t0\nN\t{bond_length}\t0\t0\n'  
    return xyz_str
    
    

In [4]:
xyz_file = build_stretched_H2O(3)
# xyz_file = build_N2(2)

viewer = Draw_molecule(xyz_file,
                       width=400,
                       height=400,
                       style="sphere")
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
print(xyz_file)

3
 
O	0	0	0
H	3	0	0
H	-0.7257656867990033	2.9108871788279895	0


In [6]:
from symred.chem import PySCFDriver

In [7]:
basis = 'sto-3g' # 'STO-3G'#
convergence = 1e-6
charge= 0
max_hf_cycles=1_000
ram = 8_000
run_mp2  = False
run_cisd = False
run_ccsd = True
run_fci  = False

In [8]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=charge,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

In [9]:
pyscf_obj.run_pyscf()

In [10]:
pyscf_obj.pyscf_hf.converged

True

In [11]:
pyscf_obj.pyscf_ccsd.converged

True

In [12]:
pyscf_obj.pyscf_ccsd.max_cycle

1000

In [13]:
2*pyscf_obj.pyscf_hf.mol.nao

14

In [14]:
from symred.chem import FermionicHamilt

In [15]:
H_ferm = FermionicHamilt(pyscf_obj.pyscf_hf)

H_ferm.build_operator()

H_ferm.fermionic_molecular_hamiltonian

() 2.934201044849797
((0, 1), (0, 0)) -31.95318959614977
((0, 1), (2, 0)) 0.6204482181474416
((0, 1), (4, 0)) -0.002715143629717858
((0, 1), (8, 0)) -0.008341402641731856
((1, 1), (1, 0)) -31.95318959614977
((1, 1), (3, 0)) 0.6204482181474416
((1, 1), (5, 0)) -0.002715143629717858
((1, 1), (9, 0)) -0.008341402641731856
((2, 1), (0, 0)) 0.6204482181474393
((2, 1), (2, 0)) -7.290321789346246
((2, 1), (4, 0)) -0.023433309621282823
((2, 1), (8, 0)) 0.044753030415396265
((3, 1), (1, 0)) 0.6204482181474393
((3, 1), (3, 0)) -7.290321789346246
((3, 1), (5, 0)) -0.023433309621282823
((3, 1), (9, 0)) 0.044753030415396265
((4, 1), (0, 0)) -0.002715143629717857
((4, 1), (2, 0)) -0.023433309621282827
((4, 1), (4, 0)) -6.799354883710101
((4, 1), (8, 0)) 0.1497678821055011
((5, 1), (1, 0)) -0.002715143629717857
((5, 1), (3, 0)) -0.023433309621282827
((5, 1), (5, 0)) -6.799354883710101
((5, 1), (9, 0)) 0.1497678821055011
((6, 1), (6, 0)) -6.800247276802284
((7, 1), (7, 0)) -6.800247276802284
((8, 1), 

In [16]:
hf_state = H_ferm.hf_comp_basis_state
hf_state


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [17]:
hf_ket = H_ferm.hf_ket
H_mat = H_ferm.get_sparse_ham()

In [18]:
from pyscf.cc.addons import spatial2spin

# doubles!

t2 = spatial2spin(pyscf_obj.pyscf_ccsd.t2)
no, nv = t2.shape[1:3]
nmo = no + nv
double_amps = np.zeros((nmo, nmo, nmo, nmo))
double_amps[no:,:no,no:,:no] = .5 * t2.transpose(2,0,3,1)

In [19]:
# singles

t1 = spatial2spin(pyscf_obj.pyscf_ccsd.t1)
no, nv = t1.shape
nmo = no + nv
ccsd_single_amps = np.zeros((nmo, nmo))
ccsd_single_amps[no:,:no] = t1.T
ccsd_single_amps.shape

(14, 14)

In [20]:
double_amplitudes_list=[]
double_amplitudes = double_amps
for i, j, k, l in zip(*double_amplitudes.nonzero()):
    if not np.isclose(double_amplitudes[i, j, k, l], 0):
        double_amplitudes_list.append([[i, j, k, l],
                                       double_amplitudes[i, j, k, l]])
    
double_amplitudes_list

[[[10, 0, 11, 1], 7.868454456630874e-07],
 [[10, 0, 11, 3], 2.018213123703696e-06],
 [[10, 0, 11, 5], 2.532142595106459e-08],
 [[10, 0, 11, 9], 1.4148158860448373e-06],
 [[10, 0, 12, 2], -3.3025733684225275e-06],
 [[10, 0, 12, 4], -1.4015243265129824e-06],
 [[10, 0, 12, 8], -1.02156921889634e-08],
 [[10, 0, 13, 1], -6.346470972259797e-06],
 [[10, 0, 13, 3], -2.1062790950856375e-05],
 [[10, 0, 13, 5], -3.877591846127424e-08],
 [[10, 0, 13, 9], 5.003073596825096e-07],
 [[10, 1, 11, 0], -7.868454456630874e-07],
 [[10, 1, 11, 2], -2.018213123703696e-06],
 [[10, 1, 11, 4], -2.532142595106459e-08],
 [[10, 1, 11, 8], -1.4148158860448373e-06],
 [[10, 1, 13, 0], 6.346470972259797e-06],
 [[10, 1, 13, 2], 1.7760217582433847e-05],
 [[10, 1, 13, 4], -1.3627484080517081e-06],
 [[10, 1, 13, 8], -5.10523051871473e-07],
 [[10, 2, 11, 1], 2.018213123703696e-06],
 [[10, 2, 11, 3], 9.591398752420751e-05],
 [[10, 2, 11, 5], -7.947662647378791e-07],
 [[10, 2, 11, 9], -0.0005123119974040205],
 [[10, 2, 12, 0

In [21]:
single_amplitudes_list = []
for i, j in zip(*ccsd_single_amps.nonzero()):
    single_amplitudes_list.append([[i, j], ccsd_single_amps[i, j]])

single_amplitudes_list

[[[10, 0], 2.396291706251513e-11],
 [[10, 2], 2.287509233108026e-09],
 [[10, 4], 4.41245353150174e-07],
 [[10, 6], 9.712424301126691e-11],
 [[10, 8], 4.868194848435929e-07],
 [[11, 1], 2.396291706251513e-11],
 [[11, 3], 2.287509233108026e-09],
 [[11, 5], 4.41245353150174e-07],
 [[11, 7], 9.712424301126691e-11],
 [[11, 9], 4.868194848435929e-07],
 [[12, 0], -7.368346916285918e-12],
 [[12, 2], -2.8552009284038475e-09],
 [[12, 4], -6.9981291660655266e-06],
 [[12, 6], -1.9406733200424525e-09],
 [[12, 8], 2.8480871940717316e-07],
 [[13, 1], -7.368346916285918e-12],
 [[13, 3], -2.8552009284038475e-09],
 [[13, 5], -6.9981291660655266e-06],
 [[13, 7], -1.9406733200424525e-09],
 [[13, 9], 2.8480871940717316e-07]]

In [22]:
generator_t2 = FermionOperator()

# Add double excitations
for (i, j, k, l), t_ijkl in double_amplitudes_list:
    i, j, k, l = int(i), int(j), int(k), int(l)
    generator_t2 += FermionOperator(((i, 1), (j, 0), (k, 1), (l, 0)), t_ijkl)
#     if anti_hermitian:
#         generator += FermionOperator(((l, 1), (k, 0), (j, 1), (i, 0)),
#                                      -t_ijkl)

In [23]:
generator_t1 = FermionOperator()
for (i, j), t_ij in single_amplitudes_list:
    i, j = int(i), int(j)
    generator_t1 += FermionOperator(((i, 1), (j, 0)), t_ij)

In [24]:
T = generator_t1 + 0.5*generator_t2

ccsd_ansatz = expm(get_sparse_operator(T, n_qubits=H_ferm.n_qubits))

In [25]:
ccsd_ket =  ccsd_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ ccsd_ket

(-74.64243117114893+0j)

In [26]:
pyscf_obj.pyscf_ccsd.e_tot 

-74.64243117121241

In [27]:
# intermediate normalization
hf_ket.conj().T  @ ccsd_ket


(1+0j)

In [28]:
normalized_state = ccsd_ket / np.sqrt(ccsd_ket.conj().T@ccsd_ket)

E_with_norm_state = normalized_state.conj().T @ H_mat @ normalized_state
E_with_norm_state

(-74.6423653685226+0j)

In [29]:
abs(E_with_norm_state-pyscf_obj.pyscf_ccsd.e_tot )

6.580268980371784e-05

In [30]:
# normalized_state = ccsd_ket / np.sqrt(ccsd_ket.conj().T@ccsd_ket)

# normalized_state.conj().T @ H_mat @ normalized_state

In [31]:
for ind, amp in enumerate(normalized_state):
    if not np.isclose(amp, 0):
        binary_string = np.binary_repr(ind, width=H_ferm.n_qubits)
        print(f'{amp}|{binary_string}>')

(-2.164168763714053e-08+0j)|00001111111111>
(4.1761628007949623e-08+0j)|00110111101111>
(-4.176162800794961e-08+0j)|00111011011111>
(-8.704342884420567e-08+0j)|00111111001111>
(-2.5436645017122316e-07+0j)|00111111110011>
(6.855815501258255e-06+0j)|00111111110110>
(-6.855815501258255e-06+0j)|00111111111001>
(8.499947808869657e-07+0j)|00111111111100>
(-1.4398651155518856e-08+0j)|01001011111111>
(3.5676258179136905e-06+0j)|01011111111010>
(4.9511001060913636e-08+0j)|01100011111111>
(1.181030671339231e-07+0j)|01100111101111>
(-1.2551992458693792e-07+0j)|01101011011111>
(3.7198445689273037e-07+0j)|01101100111111>
(-4.5684523678399733e-07+0j)|01101111001111>
(-5.875008957454686e-07+0j)|01101111110011>
(1.918558763438476e-05+0j)|01101111110110>
(-2.275321345229845e-05+0j)|01101111111001>
(2.1801875213499735e-06+0j)|01101111111100>
(-2.6741972701663383e-07+0j)|01110011101111>
(1.5140055386839493e-06+0j)|01110111111010>
(-9.67303567419962e-08+0j)|01111000111111>
(-1.040364553486166e-07+0j)|0111

In [32]:
### TEST TAPER!

In [33]:
from symred.S3_projection import QubitTapering
from openfermion import jordan_wigner, QubitOperator

In [34]:
def QubitOperator_to_dict(op, num_qubits):
    assert(type(op) == QubitOperator)
    op_dict = {}
    term_dict = op.terms
    terms = list(term_dict.keys())

    for t in terms:    
        letters = ['I' for i in range(num_qubits)]
        for i in t:
            letters[i[0]] = i[1]
        p_string = ''.join(letters)        
        op_dict[p_string] = term_dict[t]
         
    return op_dict

In [35]:
jw_ham = jordan_wigner(H_ferm.fermionic_molecular_hamiltonian)

ham_dict = QubitOperator_to_dict(jw_ham, H_ferm.n_qubits)


In [36]:
from symred.symplectic_form import PauliwordOp
ham = PauliwordOp(ham_dict)

In [37]:
taper_hamiltonian = QubitTapering(ham)

print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
print('\nand are rotated onto the single-qubit Pauli operators\n')
print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
print('\nvia a sequence of Clifford pi/2 rotations\n')
print(taper_hamiltonian.stabilizers.stabilizer_rotations[0])

We are able to taper 4 qubits from the Hamiltonian.

The symmetry generators are

1+0j ZIZIZIIZZIIZIZ +
1+0j IZIZIZIZIZIZIZ +
1+0j IIIIIIZZIIIIII +
1+0j IIIIIIIIIIZZZZ

and are rotated onto the single-qubit Pauli operators

-1+0j IIIIIIIIIIXIII +
-1+0j IIIIIIXIIIIIII +
-1+0j IXIIIIIIIIIIII +
-1+0j XIIIIIIIIIIIII

via a sequence of Clifford pi/2 rotations

('IIIIIIYZIIIIII', None)


In [38]:
taper_hamiltonian.stabilizers.update_sector(H_ferm.hf_comp_basis_state)
print(f'The symmetry sector corresponding with the reference state is {taper_hamiltonian.stabilizers.coeff_vec}')

The symmetry sector corresponding with the reference state is [-1 -1  1  1]


In [39]:
max_indices = np.argsort(abs(normalized_state))[::-1]

max_ind=max_indices[0]

binary_string = np.binary_repr(max_ind, width=H_ferm.n_qubits)
state = np.array(list(map(int, list(binary_string))), dtype=int)
state

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [40]:
normalized_state

array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])

In [41]:
taper_hamiltonian.stabilizers.update_sector(state)
print(f'The symmetry sector corresponding with the reference state is {taper_hamiltonian.stabilizers.coeff_vec}')

The symmetry sector corresponding with the reference state is [-1 -1  1  1]


In [42]:
from symred.utils import get_ground_state_sparse


true_gs_energy, true_gs_vec = get_ground_state_sparse(ham.to_sparse_matrix)

In [43]:
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_state)
tap_gs_energy, tap_gs = get_ground_state(ham_tap.to_sparse_matrix)

print(f'The ground state energy of the full system is {true_gs_energy},')
print(f'whereas for the tapered system we find the energy is {tap_gs_energy}.')
print(f'The absolute error is {tap_gs_energy-true_gs_energy}.')

NameError: name 'get_ground_state' is not defined

In [ ]:
ham_tap = taper_hamiltonian.taper_it(ref_state=state)
tap_gs_energy, tap_gs = get_ground_state(ham_tap.to_sparse_matrix)

print(f'The ground state energy of the full system is {true_gs_energy},')
print(f'whereas for the tapered system we find the energy is {tap_gs_energy}.')
print(f'The absolute error is {tap_gs_energy-true_gs_energy}.')